# Probability and Bayesian Networks

This is a model that explains the functioning of the world from the view of Artificial Intelligence (AI). The AI inputs refer to relevant issues upon the world which are considered to be risks to incur in a low level of welfare; the risk levels increases but that status can be avoided if applying AI. When applying AI, those risky situations turn into development opportunities for increasing global welfare.

<p><img src="https://1drv.ms/u/s!Ar876AhKJSKRgvMpYEJoHfNG5buvog?e=qlxMba">

Remarks:

(1) Coding has been sourced partially from aima-python (probability4e.ipynb). The structure is quite similar to probability4e.ipynb.
(2) The Python file probability.py has been modified for running purposes of the current file. (It was added the global variable: AI_network).
(3) The current file is supported by certain modules of the Python File probability.py.


## `BayesNet()`

A `BayesNet` is a graph (as in the diagram above) where each node represents a random variable, and the edges are parent&rarr;child links. You can construct an empty graph with `BayesNet()`, then add variables one at a time with the method call `.add(`*variable_name, parent_names, cpt*`)`,  where the names are strings, and each of the  `parent_names` must already have been `.add`ed.

## `Variable(`*name, cpt, parents*`)`

A random variable; the ovals in the diagram above. The value of a variable depends on the value of the parents, in a probabilistic way specified by the variable's conditional probability table (CPT). Given the parents, the variable is independent of all the other variables. 
It will only allow variables with a finite discrete domain; not continuous values. 

## `ProbDist(`*mapping*`)`<br>`Factor(`*mapping*`)`

A probability distribution is a mapping of `{outcome: probability}` for every outcome of a random variable. 
You can give `ProbDist` the same arguments that you would give to the `dict` initializer.
In a probability distribution, every value is between 0 and 1, and the values sum to 1.
A `Factor` is similar to a probability distribution, except that the values need not sum to 1. Factors
are used in the variable elimination inference method.

## `Evidence(`*mapping*`)`

A mapping of `{Variable: value, ...}` pairs, describing the exact values for a set of variables&mdash;the things we know for sure.

## `CPTable(`*rows, parents*`)`

A conditional probability table (or *CPT*) describes the probability of each possible outcome value of a random variable, given the values of the parent variables. A `CPTable` is a a mapping, `{tuple: probdist, ...}`, where each tuple lists the values of each of the parent variables, in order, and each probability distribution says what the possible outcomes are, given those values of the parents. The `CPTable` for *AI* in the diagram above would be represented as follows:

    CPTable({(T, T): .90,
             (T, F): .40,
             (F, T): .75,
             (F, F): .20},
            [GlobalWarming, FosilFuels])
            
How do you read this? Take the second row, "`(T, F): .40`". This means that when the first parent (`GlobalWarming`) is true, and the second parent (`FosilFuels`) is false, then the probability of `AI` being true is .40. Note that the .40 is an abbreviation for `ProbDist({T: .40, F: .60})`.
            
## `T = Bool(True); F = Bool(False)`

When it is used `bool` values (`True` and `False`), it became hard to read rows in CPTables, because  the columns didn't line up:

     (True, True, False, False, False)
     (False, False, False, False, True)
     (True, False, False, True, True)
     
Therefore, it is created the `Bool` class, with constants `T` and `F` such that  `T == True` and `F == False`, and now rows are easier to read:

     (T, T, F, F, F)
     (F, F, F, F, T)
     (T, F, F, T, T)
     
Here is the code for these classes:

In [5]:
from probability import BayesNode
from notebook import psource, pseudocode, heatmap
from collections import defaultdict, Counter
import itertools
import math
import random

class BayesNet(object):
    "Bayesian network: a graph of variables connected by parent links."
     
    def __init__(self): 
        self.variables = [] # List of variables, in parent-first topological sort order
        self.lookup = {}    # Mapping of {variable_name: variable} pairs
            
    def add(self, name, parentnames, cpt):
        "Add a new Variable to the BayesNet. Parentnames must have been added previously."
        parents = [self.lookup[name] for name in parentnames]
        var = Variable(name, cpt, parents)
        self.variables.append(var)
        self.lookup[name] = var
        return self
    
class Variable(object):
    "A discrete random variable; conditional on zero or more parent Variables."
    
    def __init__(self, name, cpt, parents=()):
        "A variable has a name, list of parent variables, and a Conditional Probability Table."
        self.__name__ = name
        self.parents  = parents
        self.cpt      = CPTable(cpt, parents)
        self.domain   = set(itertools.chain(*self.cpt.values())) # All the outcomes in the CPT
                
    def __repr__(self): return self.__name__
    
class Factor(dict): "An {outcome: frequency} mapping."

class ProbDist(Factor):
    """A Probability Distribution is an {outcome: probability} mapping. 
    The values are normalized to sum to 1.
    ProbDist(0.75) is an abbreviation for ProbDist({T: 0.75, F: 0.25})."""
    def __init__(self, mapping=(), **kwargs):
        if isinstance(mapping, float):
            mapping = {T: mapping, F: 1 - mapping}
        self.update(mapping, **kwargs)
        normalize(self)
        
class Evidence(dict): 
    "A {variable: value} mapping, describing what we know for sure."
        
class CPTable(dict):
    "A mapping of {row: ProbDist, ...} where each row is a tuple of values of the parent variables."
    
    def __init__(self, mapping, parents=()):
        """Provides two shortcuts for writing a Conditional Probability Table. 
        With no parents, CPTable(dist) means CPTable({(): dist}).
        With one parent, CPTable({val: dist,...}) means CPTable({(val,): dist,...})."""
        if len(parents) == 0 and not (isinstance(mapping, dict) and set(mapping.keys()) == {()}):
            mapping = {(): mapping}
        for (row, dist) in mapping.items():
            if len(parents) == 1 and not isinstance(row, tuple): 
                row = (row,)
            self[row] = ProbDist(dist)

class Bool(int):
    "Just like `bool`, except values display as 'T' and 'F' instead of 'True' and 'False'"
    __str__ = __repr__ = lambda self: 'T' if self else 'F'
        
T = bool(True)
F = bool(False)

And here are some associated functions:

In [6]:
def P(var, evidence={}):
    "The probability distribution for P(variable | evidence), when all parent variables are known (in evidence)."
    row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row]

def normalize(dist):
    "Normalize a {key: value} distribution so values sum to 1.0. Mutates dist and returns it."
    total = sum(dist.values())
    for key in dist:
        dist[key] = dist[key] / total
        assert 0 <= dist[key] <= 1, "Probabilities must be between 0 and 1."
    return dist

def sample(probdist):
    "Randomly sample an outcome from a probability distribution."
    r = random.random() # r is a random point in the probability distribution
    c = 0.0             # c is the cumulative probability of outcomes seen so far
    for outcome in probdist:
        c += probdist[outcome]
        if r <= c:
            return outcome
        
def globalize(mapping):
    "Given a {name: value} mapping, export all the names to the `globals()` namespace."
    globals().update(mapping)

# Sample Usage

Here are some examples of using the classes:

In [7]:
# Example random variable: GlobalWarming:
# Global Warming is expected to be highly risky in the long-term with a 0.6 of occurrence certainty, independent of
# any other variables.
GlobalWarming = Variable('GlobalWarming', 0.6)

In [8]:
# The probability distribution for GlobalWarming
P(GlobalWarming)

{True: 0.6, False: 0.4}

In [9]:
# Get the probability of a specific outcome by subscripting the probability distribution
P(GlobalWarming)[T]

0.6

In [10]:
# Randomly sample from the distribution:
sample(P(GlobalWarming))

True

In [11]:
# Randomly sample 100,000 times, and count up the results:
Counter(sample(P(GlobalWarming)) for i in range(100000))

Counter({False: 39835, True: 60165})

In [12]:
# Two equivalent ways of specifying the same Boolean probability distribution:
assert ProbDist(0.75) == ProbDist({T: 0.75, F: 0.25})

In [13]:
# Two equivalent ways of specifying the same non-Boolean probability distribution:
assert ProbDist(win=15, lose=3, tie=2) == ProbDist({'win': 15, 'lose': 3, 'tie': 2})
ProbDist(win=15, lose=3, tie=2)

{'win': 0.75, 'lose': 0.15, 'tie': 0.1}

In [14]:
# The difference between a Factor and a ProbDist--the ProbDist is normalized:
Factor(a=1, b=2, c=3, d=4)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [15]:
ProbDist(a=1, b=2, c=3, d=4)

{'a': 0.1, 'b': 0.2, 'c': 0.3, 'd': 0.4}

In [16]:
AI_net = (BayesNet()
    .add('FosilFuels', [], 0.95)
    .add('GlobalWarming', [], 0.6)
    .add('AI', ['FosilFuels', 'GlobalWarming'], {(T, T): 0.90, (T, F): 0.40, (F, T): 0.75, (F, F): 0.20})
    .add('RenewableEnergy', ['AI'], {T: 0.95, F: 0.35})
    .add('Traffic', ['AI'], {T: 0.85, F: 0.45}))  

In [17]:
# Make FosilFuels, GlobalWarming, etc. be global variables
globalize(AI_net.lookup) 
AI_net.variables

[FosilFuels, GlobalWarming, AI, RenewableEnergy, Traffic]

In [18]:
# Probability distribution of a FosilFuels
P(FosilFuels)

{True: 0.95, False: 0.050000000000000044}

In [19]:
# Probability of AI going off, given a FosilFuels and not an GlobalWarming:
P(AI, {FosilFuels: T, GlobalWarming: F})

{True: 0.4, False: 0.6}

In [20]:
# Where that came from: the (T, F) row of AI's CPT:
AI.cpt

{(True, True): {True: 0.9, False: 0.09999999999999998},
 (True, False): {True: 0.4, False: 0.6},
 (False, True): {True: 0.75, False: 0.25},
 (False, False): {True: 0.2, False: 0.8}}

# Outcome 1: Queriying the network

## Bayes Nets as Joint Probability Distributions

A Bayes net is a compact way of specifying a full joint distribution over all the variables in the network.  Given a set of variables {*X*<sub>1</sub>, ..., *X*<sub>*n*</sub>}, the full joint distribution is:

P(*X*<sub>1</sub>=*x*<sub>1</sub>, ..., *X*<sub>*n*</sub>=*x*<sub>*n*</sub>) = <font size=large>&Pi;</font><sub>*i*</sub> P(*X*<sub>*i*</sub> = *x*<sub>*i*</sub> | parents(*X*<sub>*i*</sub>))

For a network with *n* variables, each of which has *b* values, there are *b<sup>n</sup>* rows in the joint distribution (for example, a billion rows for 30 Boolean variables), making it impractical to explicitly create the joint distribution for large networks.  But for small networks, the function `joint_distribution` creates the distribution, which can be instructive to look at, and can be used to do inference. 

In [21]:
def joint_distribution(net):
    "Given a Bayes net, create the joint distribution over all variables."
    return ProbDist({row: prod(P_xi_given_parents(var, row, net)
                               for var in net.variables)
                     for row in all_rows(net)})

def all_rows(net): return itertools.product(*[var.domain for var in net.variables])

def P_xi_given_parents(var, row, net):
    "The probability that var = xi, given the values in this row."
    dist = P(var, Evidence(zip(net.variables, row)))
    xi = row[net.variables.index(var)]
    return dist[xi]

def prod(numbers):
    "The product of numbers: prod([2, 3, 5]) == 30. Analogous to `sum([2, 3, 5]) == 10`."
    result = 1
    for x in numbers:
        result *= x
    return result

In [22]:
# All rows in the joint distribution (2**5 == 32 rows)
set(all_rows(AI_net))

{(False, False, False, False, False),
 (False, False, False, False, True),
 (False, False, False, True, False),
 (False, False, False, True, True),
 (False, False, True, False, False),
 (False, False, True, False, True),
 (False, False, True, True, False),
 (False, False, True, True, True),
 (False, True, False, False, False),
 (False, True, False, False, True),
 (False, True, False, True, False),
 (False, True, False, True, True),
 (False, True, True, False, False),
 (False, True, True, False, True),
 (False, True, True, True, False),
 (False, True, True, True, True),
 (True, False, False, False, False),
 (True, False, False, False, True),
 (True, False, False, True, False),
 (True, False, False, True, True),
 (True, False, True, False, False),
 (True, False, True, False, True),
 (True, False, True, True, False),
 (True, False, True, True, True),
 (True, True, False, False, False),
 (True, True, False, False, True),
 (True, True, False, True, False),
 (True, True, False, True, True),


In [23]:
# Let's work through just one row of the table:
row = (F, F, F, F, F)

In [24]:
# This is the probability distribution for AI
P(AI, {FosilFuels: F, GlobalWarming: F})

{True: 0.2, False: 0.8}

In [25]:
# Here's the probability that AI is false, given the parent values in this row:
P_xi_given_parents(AI, row, AI_net)

0.8

In [26]:
# The full joint distribution:
joint_distribution(AI_net)

{(False, False, False, False, False): 0.005720000000000006,
 (False, False, False, False, True): 0.0046800000000000045,
 (False, False, False, True, False): 0.0030800000000000024,
 (False, False, False, True, True): 0.002520000000000002,
 (False, False, True, False, False): 3.000000000000006e-05,
 (False, False, True, False, True): 0.0001700000000000003,
 (False, False, True, True, False): 0.0005700000000000005,
 (False, False, True, True, True): 0.0032300000000000024,
 (False, True, False, False, False): 0.0026812500000000026,
 (False, True, False, False, True): 0.002193750000000002,
 (False, True, False, True, False): 0.0014437500000000015,
 (False, True, False, True, True): 0.001181250000000001,
 (False, True, True, False, False): 0.00016875000000000033,
 (False, True, True, False, True): 0.0009562500000000017,
 (False, True, True, True, False): 0.0032062500000000034,
 (False, True, True, True, True): 0.018168750000000015,
 (True, False, False, False, False): 0.08151,
 (True, False,

In [27]:
# Probability that "the AI begins ruling, but neither Fossil Fuels nor a Global Warming has begun to be relevant,
# and Renewable Energy, and Traffic occur."

print(AI_net.variables)
joint_distribution(AI_net)[F, F, T, T, T]

[FosilFuels, GlobalWarming, AI, RenewableEnergy, Traffic]


0.0032300000000000024

## Inference by Querying the Joint Distribution

One can use `P(variable, evidence)` to get the probability of a variable, if one knows the vaules of all the parent variables. But what if one doesn't know? Bayes nets allow to calculate the probability, but the calculation is not just a lookup in the CPT; it is a global calculation across the whole net. One inefficient but straightforward way of doing the calculation is to create the joint probability distribution, then pick out just the rows that match the evidence variables, and for each row check what the value of the query variable is, and increment the probability for that value accordningly:

In [28]:
def enumeration_ask(X, evidence, net):
    "The probability distribution for query variable X in a belief net, given evidence."
    i    = net.variables.index(X) # The index of the query variable X in the row
    dist = defaultdict(float)     # The resulting probability distribution over X
    for (row, p) in joint_distribution(net).items():
        if matches_evidence(row, evidence, net):
            dist[row[i]] += p
    return ProbDist(dist)

def matches_evidence(row, evidence, net):
    "Does the tuple of values for this row agree with the evidence?"
    return all(evidence[v] == row[net.variables.index(v)]
               for v in evidence)

Query 1

In [29]:
# Incurring in Fossil Fuels associated dangers, given that Renewable Energy begins to apply
# whereas Traffic (the improved one) is not applied: 
enumeration_ask(FosilFuels, {RenewableEnergy: F, Traffic: T}, AI_net)

{False: 0.0668756530825497, True: 0.9331243469174503}

Query 2

In [30]:
# The application probability of AI, given that Global Warming is begun to be dangerous and
# improved traffic (Traffic=True) is applied:
enumeration_ask(AI, {Traffic: T, GlobalWarming: T}, AI_net)

{False: 0.0599442379182156, True: 0.9400557620817844}

# Outcome 2: Visual depiction of the network

## BayesNodes

In [31]:
psource(BayesNode)

## AI Node

In [32]:
AI_node = BayesNode('AI', ['GlobalWarming', 'FosilFuels'], 
                       {(True, True): 0.90,(True, False): 0.40, (False, True): 0.75, (False, False): 0.20})

## Outcome nodes

In [33]:
RenewableEnergy_node = BayesNode('RenewableEnergy', ['AI'], {True: 0.95, False: 0.35})
Traffic_node = BayesNode('Traffic', 'AI', {(True, ): 0.85, (False, ): 0.45})

## Input nodes

In [34]:
GlobalWarming_node = BayesNode('GlobalWarming', '', 0.6)
FosilFuels_node = BayesNode('FosilFuels', '', 0.95)

It is possible to use the node for lookup function using the p method. The method takes in two arguments value and event. Event must be a dict of the type {variable:values, ..} The value corresponds to the value of the variable we are interested in (False or True).The method returns the conditional probability P(X=value | parents=parent_values), where parent_values are the values of parents in event. (event must assign each parent a value.)

In [35]:
Traffic_node.p(False, {'AI': True, 'FosilFuels': True}) # P(Traffic=False | FosilFuels=True)

0.15000000000000002

With all the information about nodes present it is possible to construct a Bayes Network using BayesNet. The BayesNet class does not take in nodes as input but instead takes a list of node_specs. An entry in node_specs is a tuple of the parameters we use to construct a BayesNode namely (X, parents, cpt). node_specs must be ordered with parents before children.

In [36]:
from probability import *
psource(BayesNet)

The constructor of BayesNet takes each item in node_specs and adds a BayesNode to its nodes object variable by calling the add method. add in turn adds node to the net. Its parents must already be in the net, and its variable must not. Thus add allows us to grow a BayesNet given its parents are already present.

AI_network global is an instance of BayesNet corresponding to the above example.

In [37]:
T, F = True, False

AI_network = BayesNet([
    ('GlobalWarming', '', 0.6),
    ('FosilFuels', '', 0.95),
    ('AI', 'GlobalWarming FosilFuels',
     {(T, T): 0.90, (T, F): 0.40, (F, T): 0.75, (F, F): 0.20}),
    ('RenewableEnergy', 'AI', {T: 0.95, F: 0.35}),
    ('Traffic', 'AI', {T: 0.85, F: 0.45})
])

Outcome Network with BayesNet class

In [38]:
AI_network

BayesNet([('GlobalWarming', ''), ('FosilFuels', ''), ('AI', 'GlobalWarming FosilFuels'), ('RenewableEnergy', 'AI'), ('Traffic', 'AI')])

# Outcome 3: Associated Details of the Conditional Probability Tables

BayesNet method variable_node allows to reach BayesNode instances inside a Bayes Net. It is possible to modify the cpt of the nodes directly using this method.

In [39]:
from probability import *

type(AI_network.variable_node('AI'))

probability.BayesNode

Conditional probability table (or CPT)

In [40]:
from probability import *
AI_network.variable_node('AI').cpt

{(True, True): 0.9,
 (True, False): 0.4,
 (False, True): 0.75,
 (False, False): 0.2}